## VRP

In [1]:
# paramaters
from string import ascii_letters

capacity = 5
num_customers = 10
max_customer_repeat = 2
max_journey = num_customers * max_customer_repeat - 1
customers_letters = [letter for letter in ascii_letters[:num_customers]]
max_demand=3
starting_point = (-50,100)

city_length = 700
city_width = 700

population_size = 300
# cross
margin = 2
child_length_type = "parent length" 
# mutation
length_prob = 0.001
swap_prob = 0.1
length_range = 10


num_iters = 1000
early_stopping = True
stopping_num = 10000



In [2]:
from numpy import random


def generateDemand(num_customers,max_demand):
    random.seed(7)
    demands = []
    while(True):
        possible_demands = [i for i in range(-max_demand,max_demand+1) if i !=0]
        demands = [random.choice(possible_demands) for _ in range(num_customers)]
        if (sum(demands) == 0):
            break
    return demands

random.seed(7)
demands = generateDemand(num_customers,max_demand)

customers = {lett:((random.randint(0,city_width),random.randint(0,city_length)),demand) for lett,demand in zip(customers_letters,demands)}

In [3]:

# child_length_type = "parent length"
# length_range = 10
# capacity = 0
# max_customer_repeat = 1
# max_journey = num_customers
# customers_letters = [letter for letter in ascii_letters[:num_customers]]
# max_demand=0
# random.seed(7)
# demands = [0 for _ in range(num_customers)]

# customers = {lett:((random.randint(0,city_width),random.randint(0,city_length)),demand) for lett,demand in zip(customers_letters,demands)}

## Fitness

In [4]:
def distance(d1,d2):
    return ((d1[0] - d2[0]) ** 2 + (d1[1] - d2[1]) ** 2) ** 0.5

def totalDistance(journey,with_starting):
    total_distance = 0
    if with_starting:
        total_distance = distance(starting_point,customers[journey[0]][0])
    for i in range(len(journey)-1):
        c_0 = journey[i]
        c_1 = journey[i+1]
        total_distance += distance(customers[c_0][0],customers[c_1][0])
    return total_distance



def grantedDemands(journey,return_ungranted_count=False):

    customers_demands = {letter:demand for letter, (_, demand) in customers.items()} 
    current_demand = 0

    for c in journey:
        cust_demand = customers_demands[c]
        if cust_demand>0: # if customers gives
            current_capacity = capacity - current_demand
            cust_granted_demand = min(current_capacity, cust_demand) 
        elif cust_demand<0: # if customers gives
            cust_granted_demand = max(-current_demand, cust_demand)
        else:
            continue

        customers_demands[c] = cust_demand - cust_granted_demand
        current_demand += cust_granted_demand
  
    ungranted_demand = sum([abs(demand) for demand in customers_demands.values()])
    
    return  ungranted_demand == 0 

def fitness(journey):
    return (grantedDemands(journey), totalDistance(journey,with_starting=True))

In [5]:
import random as rnd 

def sameCustomer(journey):
    indexes = []
    all_places = []
    for i in range(len(journey)-1):
        if journey[i] == journey[i+1]:
            indexes.append(i+1) 
            pla = []
            for j in range(len(journey)):
                if journey[j] == journey[i]:
                    pla.append(j)
            all_places.append(pla)

    return (indexes,all_places)

def swapCustomers(journey, i, j):
    jour_list = list(journey)
    jour_list[i], jour_list[j] = jour_list[j], jour_list[i]

    return "".join(jour_list)

def initPopulation(N,seed = None):
    if seed:
        random.seed(seed)
    population = []
    while(len(population)<N):
        journey = customers_letters + rnd.choices(customers_letters,k=random.randint(max_journey - num_customers+1))
        journey = rnd.sample(journey,len(journey))
        while(sameCustomer(journey)[0]):
            journey = rnd.sample(journey,len(journey))
        journey = "".join(journey)
        population.append(journey)
        population = list(set(population))
    return population

## Only for testing

In [6]:

def journeyIsValid(journey,return_reason = False):
    journey_valid = True
    reason = ""
    if len(journey)>max_journey or len(journey)<num_customers:
        reason = "length"
        journey_valid = False
    
    if (set(journey)!=set(customers_letters)):
        reason = "not all Visited or unknown char"
        journey_valid = False
    
    if (sameCustomer(journey)[0]):
        reason = "succeeding same customer"
        journey_valid = False
    
    for f in custHistogram(journey,customers_letters).values():
        if f>max_customer_repeat:
            reason = "a customer is visited more then maximum visits"
            journey_valid = False 
    
    if return_reason:
        return (journey_valid, reason)
    return journey_valid

#

In [7]:

def custHistogram(j,letters):
    frequencies = {c:0 for c in letters}
    for c in j:
        if c in letters:
            frequencies[c] += 1
    return frequencies
    
def maximumVisited(j,letters):
    hist = custHistogram(j,letters)
    maximum_visited = []
    for c in hist.keys():
        if hist[c]>=max_customer_repeat:
            maximum_visited.append(c)
    return maximum_visited

# this getChild function takes into account that the child should include all customers 
def getChild(current_parent,other_parent,margin=2, length_type = "random"):
    if length_type =="random":
        min_length = min(len(current_parent),len(other_parent))
        max_length = max(len(current_parent),len(other_parent))
        len_child = random.randint(min_length,max_length+1)
    else: len_child = len(current_parent)

    didnt_visit = customers_letters.copy()

    child = other_parent[0]

    if child[-1] in didnt_visit: 
        didnt_visit.remove(child[-1])
    
    for i in range(1,len_child):

        ind = i % len(current_parent)
        tries = 0

        should_visit_new =  len(didnt_visit)>0 and(len_child - len(child)) - len(didnt_visit) <= margin

        
        while(child[-1] == current_parent[ind] or (should_visit_new and current_parent[ind] not in didnt_visit)):
            ind = (ind+1) % len(current_parent)
            tries +=1
            
            if tries>len(current_parent):
                current_parent, other_parent = other_parent, current_parent
            
        child += current_parent[ind]

        if child[-1] in didnt_visit: 
            didnt_visit.remove(child[-1])

        maximum_visited = maximumVisited(child,customers_letters)
        current_parent = [c for c in current_parent if c not in maximum_visited]
        other_parent = [c for c in other_parent if c not in maximum_visited]

        current_parent, other_parent = other_parent, current_parent
    return child

# Evolution

In [8]:


def custRepeat(j,c):
    repeats = 0
    for c_i in j:
        if c_i == c: repeats +=1
    return repeats

def completeJourney(j,N):
    rest_len = N-len(j)
    for _ in range(rest_len):
        cant_use = [c for c in j if custRepeat(j,c)>=max_customer_repeat]
        cant_use.append(j[-1])
        cant_use = list(set(cant_use))
        new_c = random.choice([c for c in customers_letters if c not in cant_use])
        j += new_c
    return j


def cross(parent1,parent2,child_length_type):
    child1 = getChild(parent1, parent2,margin = margin,length_type=child_length_type)
    child2 = getChild(parent2, parent1,margin = margin,length_type=child_length_type)

    return (child1, child2)
    
def changeOrder(j,i1):
    try:
        firsts_neighs = [i1-1,i1+1]
        firsts_neighs = [i for i in firsts_neighs if i>=0 and i<len(j)]
        cant_use = [i1]
        for l, _ in enumerate(j):
            second_neighbors = [l-1,l+1]
            second_neighbors = [i for i in second_neighbors if i>=0 and i<len(j)]
            for k in firsts_neighs:
                    if k != l and j[l] == j[k]:
                            cant_use.append(l)
                            break

            if(j[i1] in [j[k] for k in second_neighbors]):
                    cant_use.append(l)
        cant_use = set(cant_use)
        i2 = random.choice([i for i,_ in enumerate(j) if i not in cant_use])
        j = swapCustomers(j, i1,i2)      
    except:
        pass
    return j


def changeLength(j, length_range):
    length_variance = random.randint(0,length_range+1)
    new_length = len(j) + length_variance * (-1) ** random.randint(1,2+1)
    new_length = max(new_length, max_journey)
    new_length = min(new_length, num_customers)
    j = completeJourney(j, new_length)
    return j

def mutate(journey,length_prob,swap_prob,length_range):
    for i,_ in enumerate(journey):
        if random.random()<swap_prob:
            journey = changeOrder(journey,i)
            
    if random.random()<length_prob:
        journey = changeLength(journey, length_range)
        

    return journey
def produce(pop,child_length_type):
    pop = rnd.sample(pop,len(pop))
    new_pop = []
    for i in range(0,len(pop),2):
        child1, child2 = cross(pop[i],pop[(i+1) % len(pop)],child_length_type)
        new_pop.append(child1)
        new_pop.append(child2)
    return new_pop
    

# Change the produce function os it make cross only between top

# Ranking

In [9]:
def rank(population):
    population_fitness = [fitness(j) for j in population]
    under_capacity = list(filter(lambda j: population_fitness[population.index(j)][0], population))
    off_capacity = [j for j in population if j not in under_capacity]

    under_capacity = sorted(under_capacity, key=lambda j: population_fitness[population.index(j)][1])
    off_capacity = sorted(off_capacity, key=lambda j: population_fitness[population.index(j)][1])
    # off_capacity = rnd.sample(off_capacity,len(off_capacity))

    off_capacity_ind = len(under_capacity)
    ranked_population = under_capacity + off_capacity
    return (ranked_population, off_capacity_ind)


In [10]:

def evolve(pop):
    new_pop = pop + produce(pop[:len(pop)//2],child_length_type="random")
    # new_pop = pop
    for i, j in enumerate(new_pop):
        new_pop[i] = mutate(j,length_prob=length_prob,swap_prob=swap_prob,length_range=length_range)
    new_pop = list(set(new_pop))
    new_pop,_ = rank(new_pop)
    return new_pop[:population_size]


In [11]:
import turtle


t = turtle.Turtle()

def getDemandsHistory(journey):
    demands_history = [0]

    customers_demands = {letter:demand for letter, (_, demand) in customers.items()} 
    current_demand = 0

    for c in journey:
        cust_demand = customers_demands[c]
        if cust_demand>0: # if customers gives
            current_capacity = capacity - current_demand
            cust_granted_demand = min(current_capacity, cust_demand) 
        elif cust_demand<0: # if customers gives
            cust_granted_demand = max(-current_demand, cust_demand)
        else:
            continue

        customers_demands[c] = cust_demand - cust_granted_demand
        current_demand += cust_granted_demand
        demands_history.append(current_demand)
  
    
    return  demands_history

def drawDots():
  
  t.color("black")
  for na,c in customers.items():
      x,y = c[0]
      t.goto(x/2-200,y/2-200)
      t.dot()
      t.write(na+str(c[1]), font=("Arial", 14, "bold"))
  t.color("blue")

def writeSE(text,x,y):
  t.speed(0)
  t.penup()
  if text == "S":
    t.goto(x-5, y+15)
  else: t.goto(x+5,y+15)
  t.pendown()
  t.color("green")
  t.write(text, font=("Arial", 14, "bold"))
  t.color("blue")
  t.penup()
  t.goto(x, y)
  t.pendown()
  t.speed(1)

def drawJourney(journey, number_of_iterations, i,with_starting_point):
  t.penup()
  t.speed(0)
  t.goto(-300,200)
  t.pendown()
  t.write(f"Number of iterations: {number_of_iterations}", font=("Arial", 16, "bold"))
  t.penup()
  drawDots()
  if with_starting_point:
    x,y = [v/2-200 for v in starting_point] 
    t.goto(x,y)
    t.pendown()


  for i, c in enumerate(journey[:-1]):
    x1,y1 = [v/2-200 for v in customers[c][0]]
    x2,y2 = [v/2-200 for v in customers[journey[i+1]][0]]
    t.goto(x1, y1)
    t.speed(1)
    t.pendown()
    if i ==0: writeSE("S",x1,y1)
   
    t.goto(x2, y2)
    if i+1 == len(journey)-1: writeSE("E",x2,y2)
  t.speed(0)
  t.penup()
  t.goto(-300,-200)
  finish, distance = fitness(journey)
  t.color("black")
  t.write(f"Journey {i}: {journey}, Fitness: ({finish}, {distance:.2f})", font=("Arial", 16, "bold"))
  demands_history = getDemandsHistory(journey)
  t.goto(-300,-250)
  t.write(f"vehicle content: {demands_history}", font=("Arial", 16, "bold"))
  t.color("blue")

def clear():
  t.clear()
  t.reset()
  t.penup()

In [12]:
pop_0 = initPopulation(population_size)
pop_0, _ = rank(pop_0)

# Run

In [13]:
pop = pop_0.copy()
bests = []
best = (pop[0], fitness(pop[0]),0)
for i in range(num_iters):
    if (i / num_iters) % (1/100) ==0 :
        print(best)
    bests.append((pop[0], fitness(pop[0]),i))
    best_j, _ = rank([best[0],pop[0]])
    best_j = best_j[0]
    best = best if best[0]==best_j else (pop[0],fitness(pop[0]),i)
    pop = evolve(pop)

    if early_stopping:
        if i - best[2]>stopping_num:
            break 
print(best)


('hgfdeijagbc', (True, 2774.24985515066), 0)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)
('bhchdfjeigfa', (True, 2325.257801848308), 7)


In [14]:
with open("record.txt","w") as f:
    f.writelines(map(lambda b:str(b),bests))

In [ ]:
import time

best = bests[0]
for ind, j in enumerate(bests):
    tenth_len = (len(bests) // 4)
    if ind % tenth_len == 0:
        clear()
        drawJourney(best[0],ind,best[2],with_starting_point=True)
        time.sleep(1)
    if j[1][0] and j[1][1]<best[1][1]: best = j
    

clear()
drawJourney(best[0],ind,best[2],with_starting_point=True)
turtle.done()